In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
gameGames = pd.read_csv('../dataset/content_base/Steam_Games.csv', usecols=['appid', 'name', 'developer', 'categories','steamspy_tags'])


In [3]:
#Aradaki noktalı virgülleri silmek için kullanılacak method
def clean_data(x):
    if isinstance(x, str):
        return x.replace(";", "")
    else:
        print(x)
        return x

In [4]:
# steamspy_tags ve categories sütunlarında her bir tür arasındaki ';'ları sildik. Bu sayede kategorisi ve oynama özelliği birebir aynı olanları önerecek
gameGames.loc[:, 'steamspy_tags'] = gameGames['steamspy_tags'].apply(clean_data)
gameGames.loc[:, 'categories'] = gameGames['categories'].apply(clean_data)


In [5]:
#Burada benzerliklerine 1 - 0 arasında derecelendirildi
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(gameGames['steamspy_tags'].values.astype('U'))


In [6]:
#Cosine benzerliği ile oyunlar arasındaki benzerlikler çıkartıldı
#Oyunlar arasındaki benzerliklere göre sıralandı
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {}
for idx, row in gameGames.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], gameGames['appid'][i]) for i in similar_indices]
    results[row['appid']] = similar_items[1:]


In [7]:
def item(id):
    return gameGames.loc[gameGames['appid'] == id]['name'].tolist()[0].split(' - ')[0]


In [8]:
#Oyunları veri tabanına atabilmek için bir list'e atandı.
liste = []

for i in range(27075):
    liste.append((gameGames['appid'][i], gameGames['name'][i]))
    i = i+1

liste

[(10, 'Counter-Strike'),
 (20, 'Team Fortress Classic'),
 (30, 'Day of Defeat'),
 (40, 'Deathmatch Classic'),
 (50, 'Half-Life: Opposing Force'),
 (60, 'Ricochet'),
 (70, 'Half-Life'),
 (80, 'Counter-Strike: Condition Zero'),
 (130, 'Half-Life: Blue Shift'),
 (220, 'Half-Life 2'),
 (240, 'Counter-Strike: Source'),
 (280, 'Half-Life: Source'),
 (300, 'Day of Defeat: Source'),
 (320, 'Half-Life 2: Deathmatch'),
 (340, 'Half-Life 2: Lost Coast'),
 (360, 'Half-Life Deathmatch: Source'),
 (380, 'Half-Life 2: Episode One'),
 (400, 'Portal'),
 (420, 'Half-Life 2: Episode Two'),
 (440, 'Team Fortress 2'),
 (500, 'Left 4 Dead'),
 (550, 'Left 4 Dead 2'),
 (570, 'Dota 2'),
 (620, 'Portal 2'),
 (630, 'Alien Swarm'),
 (730, 'Counter-Strike: Global Offensive'),
 (1002, 'Rag Doll Kung Fu'),
 (1200, 'Red Orchestra: Ostfront 41-45'),
 (1250, 'Killing Floor'),
 (1300, 'SiN Episodes: Emergence'),
 (1500, 'Darwinia'),
 (1510, 'Uplink'),
 (1520, 'DEFCON'),
 (1530, 'Multiwinia'),
 (1600, 'Dangerous Waters')

In [9]:
#Veri tabanı tanımlandı
import sqlite3
con = sqlite3.connect('gamedatabase.db')
cur = con.cursor()

In [19]:
#Oyunların veri tabanına atımı sağlandı. Veri tabanından bir veri silindiği zaman sqlite 'database is locked' hatası döndürüyor
#Bunu önlemek için aynı tipte bir tablo oluşturulması gerekiyor. Bir sorun olursa tabloyu sil ve yeni bir aynı tablodan tekrar oluştur
cur = con.cursor()

for i in range(27075):
    cur.execute("""INSERT INTO games VALUES (?, ?)""", (int(liste[i][0]),liste[i][1]))

# Değişikleri kaydettik
con.commit()

# Bağlantıyı kapatmak zorundayız
con.close()

OperationalError: database is locked

In [70]:
# Oyun önerisi yapıp tabloya aktaracağız burada. Veri tabanı tablosu ismi recommend
# Bu tabloya verilerimizi kaydediyoruz.
# Aynı sorun burada da geçerli eğer önceden tablodan veri silinmiş ise tekrar kayıt olmada sorun yaşanıyor.
def recommend(item_id, num):
    #print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    #print("-------")
    recs = results[item_id][:num]
    durmus = []
    for rec in recs:
        #print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")
        durmus.append(item(rec[1]))
    
    con = sqlite3.connect('gamedatabase.db')
    cur = con.cursor()
    cur.execute("INSERT INTO recommend VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (durmus[0],durmus[1],
                                                                                   durmus[2],durmus[3],
                                                                                   durmus[4],durmus[5],
                                                                                   durmus[6],durmus[7],
                                                                                   durmus[8],durmus[9],item_id))
    con.commit()
    con.close()
    
        
    



In [81]:
# Methodumuza oyun appid ve kaç tane öneri istediğimizi yazıyoruz.
for i in range(0,27075):
    recommend(int(liste[i][0]), num=10)
